# Sample GeoEDF Hydrology Workflow

This notebook demonstrates a GeoEDF hydrologic workflow that acquires data (in HDF format) from a NASA Distributed Active Archive Center (DAAC) and aggregates the data across a provided watershed region. This is often the first step before running a hydrologic model. This workflow uses a pre-existing data connector (**NASAInput**) and processor (**HDFEOSShapefileMask**). 

Later we demonstrate how a new connector or processor can be built and tested in combination with this workflow before contributing it to the official repositories.

## Step 1: Run a workflow consisting of pre-existing GeoEDF plugins

Take a look at the file ``/home/scitech/shared-data/sample-geoedf-wf/mcd15.yml``. This workflow will be processed and executed using GeoEDF.

In [ ]:
from geoedfengine.GeoEDFWorkflow import GeoEDFWorkflow

In [ ]:
workflow = GeoEDFWorkflow('/home/scitech/shared-data/sample-geoedf-wf/mcd15-simple.yml')

In [ ]:
workflow.execute()

## Extending this workflow

While the workflow produced a shapefile at the end, we were not able to verify or visualize the result. 
There are Python mapping libraries (e.g. *Folium* or *ipyLeaflet*) that work with geospatial files, but require vector 
data to be in the GeoJSON format.

We will develop a new processor for converting a shapefile into a GeoJSON file and add it to the end of this workflow.

## Step 2: Build Singularity container for a new processor

A new processor is being developed with the name ``Shapefile2GeoJSON``. You can build a local Singularity image for this new plugin by following these steps:
1. Create the Singularity recipe file using the ``hpccm`` utility.
2. Build the Singularity container image (named appropriately) using the recipe file.
3. Copy the container image to the standard ``/images`` path where GeoEDF will find it.

These commands have been helpfully added to a script at ``/home/scitech/shared-data/build-local-image.sh``. 

In [ ]:
!/home/scitech/shared-data/build-local-image.sh

## Step 3: Rerun the workflow with this new processor

We can now re-run the workflow with this new processor added at the end. The GeoJSON file produced at the end can be visualized directly using the *Folium* library.

In [ ]:
from geoedfengine.GeoEDFWorkflow import GeoEDFWorkflow

In [ ]:
workflow = GeoEDFWorkflow('/home/scitech/shared-data/sample-geoedf-wf/mcd15-viz.yml')

In [ ]:
workflow.execute()

In [ ]:
import geopandas as gpd
import folium

output_dir = '/home/scitech/shared-data/sample-geoedf-wf/1600616244/output'
geojson_path = '%s/MCD15A3H.A2002197.h09v07.006.2015149103156.hdf.json' % output_dir
geo_df = gpd.GeoDataFrame.from_file(geojson_path)
field_name = 'Lai_500m'
mcd_df = geo_df.loc[:,[field_name,'geometry']]
mcd_df['id'] = mcd_df.index

test_map = folium.Map(location=[40,-86],zoom_start=7)

test_map.choropleth(geo_data=mcd_df.to_json(),
                    data=mcd_df,
                    columns=['id',field_name],
                    key_on='feature.properties.{}'.format('id'),
                    legend_name = field_name,
                    fill_color='YlOrRd',
                    fill_opacity=0.5,
                    line_weight=2)
test_map